<a href="https://colab.research.google.com/github/DIFACQUIM/Art-Driven-by-Visual-Representations-of-Chemical-Space-/blob/main/FLAVORS_LIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
      # Install rdkit
      !pip -q install rdkit
      pbar.update(100)
import pandas as pd
import numpy as np
from sys import argv

  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
# For save files at drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Load datasets

In [3]:
# Read FooDB
url = "https://raw.githubusercontent.com/DIFACQUIM/Art-Driven-by-Visual-Representations-of-Chemical-Space-/main/Datasets/FOODB_CURATED_21APRIL2023.csv"
FOODB = pd.read_csv(url)
FOODB.shape

(52856, 5)

In [4]:
# Read flavor
url = "https://raw.githubusercontent.com/DIFACQUIM/Art-Driven-by-Visual-Representations-of-Chemical-Space-/main/Datasets/Flavor.csv"
FLAVOR = pd.read_csv(url)
FLAVOR.head(2)

,id,name,flavor_group,category,created_at,updated_at,creator_id,updater_id
0,1,celery,vegetable,odor,2011-10-02 06:10:04 UTC,2011-10-02 06:10:04 UTC,NaN,NaN
1,2,corn,vegetable,odor,2011-10-02 06:10:04 UTC,2011-10-02 06:10:04 UTC,NaN,NaN


In [5]:
# Read compouns flavor
url = "https://raw.githubusercontent.com/DIFACQUIM/Art-Driven-by-Visual-Representations-of-Chemical-Space-/main/Datasets/CompoundsFlavor.csv"
COMPOUNDS_FLAVOR = pd.read_csv(url)
COMPOUNDS_FLAVOR.head(2)

,id,compound_id,flavor_id,citations,created_at,updated_at,creator_id,updater_id,source_id,source_type
0,1,11947,159,"# Arn, H, Acree TE. “Flavornet: A database of ...",2011-10-02 06:30:05 UTC,2015-10-23 23:29:45 UTC,NaN,NaN,11947,Compound
1,2,8298,213,"# Arn, H, Acree TE. “Flavornet: A database of ...",2011-10-02 06:30:05 UTC,2015-10-23 23:29:56 UTC,NaN,NaN,8298,Compound


## 2. Select columns and rename

### 2.1. FooDB

In [6]:
# Select columns
FOODB = FOODB [['ID','NAME','NEW_SMILES']]
# Rename columns
FOODB.columns =['COMPOUND_ID', 'NAME_FOOD_DB', 'NEW_SMILES']
FOODB.shape

(52856, 3)

### 2.2. Compounds flavor

In [7]:
# Select columns
COMPOUNDS_FLAVOR = COMPOUNDS_FLAVOR [['compound_id', 'flavor_id']]
# Rename columns
COMPOUNDS_FLAVOR.columns =['COMPOUND_ID', 'FLAVOR_ID']
COMPOUNDS_FLAVOR.shape

(11775, 2)

### 2.3. Flavor

In [8]:
# Rename columns
# SELECCIONAR LAS COLUMNAS
FLAVOR = FLAVOR [['id', 'name', 'flavor_group']]
# Rename columns
FLAVOR.columns =['FLAVOR_ID', 'NAME_FLAVOR', 'FLAVOR_GROUP']
FLAVOR.shape

(883, 3)

## 3. Generate dataframe with flavor_groups

In [9]:
# Merge flavor and compounds flavors
results = pd.merge(FLAVOR,COMPOUNDS_FLAVOR, how="right", on=['FLAVOR_ID'])
results.shape

(11775, 4)

In [10]:
results.head(2)

,FLAVOR_ID,NAME_FLAVOR,FLAVOR_GROUP,COMPOUND_ID
0,159,fish,NaN,11947
1,213,pungent,NaN,8298


In [11]:
# Drop NA values
results = results.dropna()
results.shape

(6474, 4)

In [12]:
# Merge flavor and compounds flavors with FooDB compounds
results = pd.merge(results,FOODB, how="right", on=['COMPOUND_ID'])
results.shape

(56179, 6)

In [13]:
# Drop NA values
results = results.dropna()
results.shape

(4964, 6)

In [14]:
results

,FLAVOR_ID,NAME_FLAVOR,FLAVOR_GROUP,COMPOUND_ID,NAME_FOOD_DB,NEW_SMILES
130,93.0,alcohol,chemical,358,4-Ethylphenol,CCc1ccc(O)cc1
131,105.0,medicinal,medicinal,358,4-Ethylphenol,CCc1ccc(O)cc1
141,30.0,lavender,floral,373,Prenyl acetate,CC(=O)OCC=C(C)C
142,44.0,sweet,balsamic,373,Prenyl acetate,CC(=O)OCC=C(C)C
143,71.0,banana,fruity,373,Prenyl acetate,CC(=O)OCC=C(C)C
...,...,...,...,...,...,...
23851,44.0,sweet,balsamic,30905,coumarin,O=c1ccc2ccccc2o1
23915,77.0,grape,fruity,31067,hexan-3-one,CCCC(=O)CC
23966,14.0,cheese,fatty,31194,n-butanoate,CCCC(=O)O
23978,93.0,alcohol,chemical,31207,n-propanol,CCCO


In [15]:
# Save file
results.to_csv("FLAVOR_CATEGORIES_27APRIL2023.csv", sep=",", index=False)